In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install traker

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn as nn
from torchvision.datasets import Food101, ImageFolder
from trak import TRAKer
from trak import modelout_functions
from collections.abc import Iterable

from src.train import train_model
from src.early_stopping import EarlyStopping


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

In [3]:
def preprocess_image(image):
    image = transforms.functional.pil_to_tensor(image)
    processed_image = processor.preprocess(image)["pixel_values"][0]
    return torch.from_numpy(processed_image)

num_classes = 10

train_dataset = Food101("data/food-101", split="train", transform=preprocess_image, download=False)
test_dataset = Food101("data/food-101", split="test", transform=preprocess_image, download=False)

In [4]:
filtered_dataset = ImageFolder("data/food-101/food-101/images", transform=preprocess_image)

chosen_indices = [*range(num_classes)] 

filtered_dataset.classes = [filtered_dataset.classes[i] for i in chosen_indices]
filtered_dataset.class_to_idx = {k: i for i, k in enumerate(filtered_dataset.classes)}

print(len(filtered_dataset))

filtered_dataset.samples = list(filter(lambda s: s[1] in chosen_indices, filtered_dataset.samples))

filtered_train_subset, filtered_test_subset = torch.utils.data.random_split(filtered_dataset, [0.8, 0.2], torch.Generator().manual_seed(42))

train_dl = torch.utils.data.DataLoader(filtered_train_subset, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(filtered_test_subset, batch_size=64, shuffle=True)

print(len(filtered_dataset))

101000
10000


In [5]:
model.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=512, out_features=num_classes))
for param in model.classifier.parameters():
        param.requires_grad = True

model.num_labels = num_classes

In [6]:
num_epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

train_model(model, train_dl, test_dl, num_epochs, optimizer, early_stopping=early_stopping)

100%|██████████| 32/32 [00:23<00:00,  1.37it/s]


[Epoch 1] Loss: 3.937, Train Acc: 0.675,Valid loss: 1.105 Valid Acc: 0.625


100%|██████████| 32/32 [00:22<00:00,  1.41it/s]


[Epoch 2] Loss: 2.393, Train Acc: 0.801,Valid loss: 1.425 Valid Acc: 0.636


100%|██████████| 32/32 [00:23<00:00,  1.39it/s]


...

[Epoch 4] Loss: 1.267, Train Acc: 0.891,Valid loss: 1.309 Valid Acc: 0.645


100%|██████████| 32/32 [00:49<00:00,  1.54s/it]


[Epoch 5] Loss: 0.925, Train Acc: 0.921,Valid loss: 1.143 Valid Acc: 0.684


100%|██████████| 32/32 [00:21<00:00,  1.47it/s]

[Epoch 6] Loss: 0.705, Train Acc: 0.939,Valid loss: 1.132 Valid Acc: 0.714
Early stopping at epoch 6


In [7]:
torch.save(model.state_dict(), "model_finetuned_baseline.pth")

In [6]:
#finetuned_path = '/content/drive/MyDrive/automating_science/model_finetuned_baseline.pth' #model_finetuned_baseline.pth
finetuned_path = "model_finetuned_baseline.pth"
checkpoint = torch.load(finetuned_path,  map_location=device)

In [7]:
#model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

#model.classifier = nn.Sequential(
#                    nn.Flatten(start_dim=1, end_dim=-1),
#                   nn.Linear(in_features=512, out_features=num_classes))

#model.num_labels = num_classes

model.load_state_dict(checkpoint)

<All keys matched successfully>

In [8]:
model = model.to(device)

In [9]:
train_dl_no_shuffle = torch.utils.data.DataLoader(filtered_train_subset, batch_size=32, shuffle=False)
test_dl_no_shuffle = torch.utils.data.DataLoader(filtered_test_subset, batch_size=32, shuffle=False)

In [10]:
class ResNetOutput(modelout_functions.AbstractModelOutput):
    def __init__(self, loss_temperature: float = 1.0):
       super().__init__()
       self.softmax = nn.Softmax(dim=-1)
       self.loss_temperature = loss_temperature

    @staticmethod
    def get_output(
                model: torch.nn.Module,
                weights: Iterable[torch.Tensor],
                buffers: Iterable[torch.Tensor],
                image: torch.Tensor,
                label: torch.Tensor
      ):
      for key, value in weights.items():
        weights[key] = weights[key].to(device)
      output = torch.func.functional_call(model, (weights, buffers), image.unsqueeze(0))
      logits = output.logits #our change
      bindex = torch.arange(logits.shape[0]).to(logits.device, non_blocking=False)
      logits_correct = logits[bindex, label.unsqueeze(0)]

      cloned_logits = logits.clone()

      cloned_logits[bindex, label.unsqueeze(0)] = torch.tensor(-torch.inf, device=logits.device, dtype=logits.dtype)

      margins = logits_correct - cloned_logits.logsumexp(dim=-1)
      return margins.sum()
    
    def get_out_to_loss_grad(self, model, weights, buffers, batch):
      for key, value in weights.items():
        weights[key] = weights[key].to(device)
      images, labels = batch
      output = torch.func.functional_call(model, (weights, buffers), images)
      logits = output.logits #our change

      ps = self.softmax(logits / self.loss_temperature)[torch.arange(logits.size(0)), labels]
      return (1 - ps).clone().detach().unsqueeze(-1)

In [11]:
traker = TRAKer(model=model,
                task=ResNetOutput(),
                train_set_size=len(train_dl_no_shuffle.dataset))

ERROR:TRAK:Could not use CudaProjector.
Reason: No module named 'fast_jl'
ERROR:TRAK:Defaulting to BasicProjector.


INFO:STORE:No existing model IDs in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.
INFO:STORE:No existing TRAK scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results.


In [13]:
model_id = 0
traker.load_checkpoint(checkpoint, model_id=0)

In [14]:
for data in tqdm(train_dl_no_shuffle):
    data = [xy.cuda() for xy in data]

    traker.featurize(batch=data, num_samples=data[0].shape[0])

  0%|          | 0/250 [00:00<?, ?it/s]c:\Users\kamil\anaconda3\envs\ml_gnn\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 250/250 [21:34<00:00,  5.18s/it]


In [15]:
traker.finalize_features()

Finalizing features for all model IDs..: 100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


In [16]:
traker.start_scoring_checkpoint(exp_name="test", checkpoint=checkpoint, model_id=model_id, num_targets=len(train_dl_no_shuffle.dataset))
for batch in tqdm(train_dl_no_shuffle):
    batch = [xy.cuda() for xy in batch]
    traker.score(batch=batch, num_samples=batch[0].shape[0])

scores = traker.finalize_scores(exp_name="test")

Finalizing scores for all model IDs..: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]
INFO:STORE:Saving scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results\scores/test.mmap


In [17]:
scores.shape

(8000, 8000)

In [18]:
traker.start_scoring_checkpoint(exp_name="test_val", checkpoint=checkpoint, model_id=model_id, num_targets=len(test_dl_no_shuffle.dataset))
for batch in tqdm(test_dl_no_shuffle):
    batch = [xy.cuda() for xy in batch]
    traker.score(batch=batch, num_samples=batch[0].shape[0])

test_scores = traker.finalize_scores(exp_name="test_val")

 98%|█████████▊| 62/63 [39:03<00:35, 35.80s/it]c:\Users\kamil\anaconda3\envs\ml_gnn\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Finalizing scores for all model IDs..: 100%|██████████| 1/1 [00:00<00:00,  7.35it/s]
INFO:STORE:Saving scores in C:\Users\kamil\OneDrive\Pulpit\przedmioty\semestr 8\automating science\trak-for-automating-science\trak_results\scores/test_val.mmap


In [19]:
test_scores.shape

(8000, 2000)

In [23]:
import numpy as np

np.save("train_scores.npy", scores)
np.save("test_scores.npy", test_scores)